# Parsing a CSGO demofile¶
##### Last Updated: August 29, 2021

The csgo package allows for the parsing of CSGO demofiles into JSON or tabular format, which makes for easy analysis. To install the package, clone the Github repository, navigate to the root directory and run `python setup.py install`. Make sure you have installed at least [Golang](https://golang.org/) version 1.14.

##### What is a demofile?
Each CSGO match typically generates recordings of the game called a demofile. Every map in a match will generate its own demofile. This demofile contains a serialization of the data transferred between the host (game server) and its clients (players). Data transferred to and from the server occurs at a predefined tick rate, which defines when client inputs are resolved with the server. For professional games, the server tick rate is usually 128 ticks per second, meaning each tick represents around 7.8 milliseconds. Client inputs represent player actions, such as movement, attacks or grenade throws. Non-client events also register in the demofile, such as round starts and ends.

Since CSGO demofiles are essentially data transferred between the clients and the game server, the data is simply stored as a text of sequential set of events with no contextual information, such as round or map location. Thus, due to the highly unstructured nature of these low level CSGO data streams, akin to log files, performing any complex analytic tasks becomes impossible without modeling this data formally into a useful data structure. The csgo package addresses this need.

##### How do I get demofiles?¶
One of the most common ways to obtain demofiles is through the site [HLTV](https://hltv.org). If you navigate to a match page, an hour or so after the match has concluded, it will often contain a link to the demofiles for that match. The link is a `.rar` compressed directory containing a demo for each map in the match.

# Initializing the parser
Let's consider the demo from a match between OG and Natus Vincere (NaVi), which we can [download from HLTV](https://www.hltv.org/matches/2344822/og-vs-natus-vincere-blast-premier-fall-series-2020). If we download the compressed demofile directory (a few hundred MB), there are three files:

- `og-vs-natus-vincere-m1-dust2.dem`
- `og-vs-natus-vincere-m2-mirage.dem`
- `og-vs-natus-vincere-m3-inferno.dem`

In order to parse one of the demofiles, you have to instantiate a DemoParser object and pass it the path to the demofile. The parser can automatically infer a `demo_id` from the filename, but you can also specify one using the argument. This name is use to title the JSON output file from the parser. We will use a `demo_id` of `OG-NaVi-BLAST2020`. 

There are a few numeric arguments to specify as one. The primary one is the `parse_rate`, or, how often do we want to parse a frame? This does _not_ affect how often we parse events, like kills or damages, only player positioning. Setting a parse rate of 1 means we are acquiring every frame (this is _very_ slow). Usually, 128 or 64 will work, depending on the tick rate of the demo. You can also specify the trade time (how many seconds inbetween trades) using the `trade_time` argument. Usually, a few seconds is good -- the default is 5. Finally, you can specify the round buy style (either "csgo" or "hltv", default is "hltv"). This argument changes the string for the round buy type output.

The parser writes to disk a file named `demoId_mapName.json`.

In [1]:
from csgo.parser import DemoParser

# Create parser object
# Set log=True above if you want to produce a logfile for the parser
demo_parser = DemoParser(demofile = "og-vs-natus-vincere-m1-dust2.dem", demo_id = "OG-NaVi-BLAST2020", parse_rate=128, log=False)


# Parse the demofile, output results to dictionary
data = demo_parser.parse()

20:43:39 [INFO] Go version>=1.14.0
20:43:39 [INFO] Initialized CSGODemoParser with demofile /home/peter/Downloads/csgo-nb/og-vs-natus-vincere-m1-dust2.dem
20:43:39 [INFO] Setting demo id to OG-NaVi-BLAST2020
20:43:39 [INFO] Setting parse rate to 128
20:43:39 [INFO] Setting trade time to 5
20:43:39 [INFO] Setting buy style to hltv
20:43:39 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.4/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
20:43:39 [INFO] Looking for file at /home/peter/Downloads/csgo-nb/og-vs-natus-vincere-m1-dust2.dem
20:44:01 [INFO] Wrote demo parse output to OG-NaVi-BLAST2020.json
20:44:01 [INFO] Reading in JSON from OG-NaVi-BLAST2020.json
20:44:01 [INFO] JSON data loaded, available in the `json` attribute to parser
20:44:01 [INFO] Successfully parsed JSON output
20:44:01 [INFO] Successfully returned JSON output


# Accessing the parsed data
Next, we can access our parsed data. The default data output is the JSON-like format of a Python dictionary. We can see the structure below:

```
├── MatchId (equal to demo_id)
├── ClientName
├── MapName
├── TickRate
├── PlayBackTicks
├── ParseRate
└── GameRounds (list of game rounds)
    ├── ... round metadata ...
    ├── Kills (list of events)
    ├── Damages (list of events)
    ├── Grenades (list of events)
    ├── BombEvents (list of events)
    ├── WeaponFires (list of events)
    ├── Flashes (list of events)
    └── Frames (list of events)
        ├── ... frame metadata ...
        ├── T
        │   ├── ... T metadata ...
        │   └── Players (list of player information)
        ├── CT (like T)
        └── World (list of objects like bombs, grenades)
```

In [2]:
for k in data.keys():
    if k != "GameRounds":
        print(k + ": " + str(data[k]))

MatchID: OG-NaVi-BLAST2020
ClientName: GOTV Demo
MapName: de_dust2
TickRate: 127
PlaybackTicks: 466670
ParseRate: 128
TradeTime: 5
RoundBuyStyle: hltv
ServerVars: {'CashBombDefused': 0, 'CashBombPlanted': 0, 'CashTeamTWinBomb': 0, 'CashWinDefuse': 3500, 'CashWinTimeRunOut': 0, 'CashWinElimination': 0, 'CashPlayerKilledDefault': 0, 'CashTeamLoserBonus': 0, 'CashTeamLoserBonusConsecutive': 0, 'RoundTime': 2, 'RoundRestartDelay': 5, 'FreezeTime': 20, 'BuyTime': 20, 'BombTimer': 0, 'MaxRounds': 30, 'TimeoutsAllowed': 4, 'CoachingAllowed': 1}


## Event Data
The parser contains data on kills, damages, grenades, bomb events, weapon fires and flashes, which we show below, in order.

In [3]:
for k in data["GameRounds"][0]["Kills"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Kills"][0][k]))

Tick: 39106
Second: 16.244094488188978
AttackerSteamID: 76561198016432560
AttackerName: mantuu
AttackerTeam: OGEsports
AttackerSide: CT
AttackerX: -294.1557312011719
AttackerY: 626.77783203125
AttackerZ: 1.0166358947753906
AttackerAreaID: 1713
AttackerAreaName: TopofMid
AttackerViewX: 319.5758056640625
AttackerViewY: 0.5987548828125
VictimSteamID: 76561198146207066
VictimName: Boombl4
VictimTeam: Natus Vincere
VictimSide: T
VictimX: 66.37433624267578
VictimY: 308.03125
VictimZ: 0.4415321350097656
VictimAreaID: 67
VictimAreaName: TopofMid
VictimViewX: 128.1170654296875
VictimViewY: 1.6534423828125
AssisterSteamID: None
AssisterName: None
AssisterTeam: None
AssisterSide: None
IsSuicide: False
IsTeamkill: False
IsWallbang: False
PenetratedObjects: 0
IsFirstKill: True
IsHeadshot: True
VictimBlinded: True
AttackerBlinded: False
FlashThrowerSteamID: 76561198013243326
FlashThrowerName: Aleksib
FlashThrowerTeam: OGEsports
FlashThrowerSide: CT
NoScope: False
ThruSmoke: False
Distance: 481.22930

In [4]:
for k in data["GameRounds"][0]["Damages"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Damages"][0][k]))

Tick: 39015
Second: 15.527559055118111
AttackerSteamID: 76561198016432560
AttackerName: mantuu
AttackerTeam: OGEsports
AttackerSide: CT
AttackerX: -284.76715087890625
AttackerY: 632.74462890625
AttackerZ: 1.266963243484497
AttackerAreaID: 1713
AttackerAreaName: TopofMid
AttackerViewX: 314.307861328125
AttackerViewY: 0.50537109375
AttackerStrafe: False
VictimSteamID: 76561198146207066
VictimName: Boombl4
VictimTeam: Natus Vincere
VictimSide: T
VictimX: 48.46458435058594
VictimY: 308.03125
VictimZ: 0.4415321350097656
VictimAreaID: 67
VictimAreaName: TopofMid
VictimViewX: 39.495849609375
VictimViewY: 1.944580078125
Weapon: USP-S
HpDamage: 32
HpDamageTaken: 32
ArmorDamage: 0
ArmorDamageTaken: 0
HitGroup: Neck


In [5]:
for k in data["GameRounds"][0]["Grenades"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Grenades"][0][k]))

Tick: 38933
Second: 14.881889763779528
ThrowerSteamID: 76561198013243326
ThrowerName: Aleksib
ThrowerTeam: OGEsports
ThrowerSide: CT
ThrowerX: -479.7312927246094
ThrowerY: 1835.4271240234375
ThrowerZ: -123.54253387451172
ThrowerAreaID: 5220
ThrowerAreaName: MidDoors
GrenadeType: Flashbang
GrenadeX: -148.5625
GrenadeY: 509.0625
GrenadeZ: 91.125
GrenadeAreaID : 8109
GrenadeAreaName: TopofMid


In [6]:
for k in data["GameRounds"][6]["BombEvents"][0].keys():
        print(k + ": " + str(data["GameRounds"][6]["BombEvents"][0][k]))

Tick: 129153
Second: 64.98425196850394
PlayerSteamID: 76561198034202275
PlayerName: s1mple
PlayerTeam: Natus Vincere
PlayerX: 1089.201904296875
PlayerY: 2489.580810546875
PlayerZ: 96.02377319335938
BombAction: plant_begin
BombSite: A


In [7]:
for k in data["GameRounds"][0]["WeaponFires"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["WeaponFires"][0][k]))

Tick: 37440
Second: 3.125984251968504
PlayerSteamID: 76561198121220486
PlayerName: Perfecto
PlayerTeam: Natus Vincere
PlayerSide: T
PlayerX: -503.520263671875
PlayerY: -661.4535522460938
PlayerZ: 127.1343002319336
PlayerAreaID: 2000
PlayerAreaName: TSpawn
PlayerViewX: 84.2376708984375
PlayerViewY: 354.0948486328125
PlayerStrafe: False
Weapon: Smoke Grenade


In [8]:
for k in data["GameRounds"][0]["Flashes"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Flashes"][0][k]))

Tick: 38933
Second: 14.881889763779528
AttackerSteamID: 76561198013243326
AttackerName: Aleksib
AttackerTeam: OGEsports
AttackerSide: CT
AttackerX: -490.8686218261719
AttackerY: 1940.558349609375
AttackerZ: -122.56611633300781
AttackerAreaID: 5228
AttackerAreaName: MidDoors
AttackerViewX: 187.503662109375
AttackerViewY: 5.91064453125
PlayerSteamID: 76561198013243326
PlayerName: Aleksib
PlayerTeam: OGEsports
PlayerSide: CT
PlayerX: -490.8686218261719
PlayerY: 1940.558349609375
PlayerZ: -122.56611633300781
PlayerAreaID: 5228
PlayerAreaName: MidDoors
PlayerViewX: 187.503662109375
PlayerViewY: 5.91064453125
FlashDuration: 1.086047488


## Event Data as DataFrames

We can also parse the data into dataframes.

In [9]:
df = demo_parser.parse(return_type="df")
df.keys()

20:44:01 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.4/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
20:44:01 [INFO] Looking for file at /home/peter/Downloads/csgo-nb/og-vs-natus-vincere-m1-dust2.dem
20:44:23 [INFO] Wrote demo parse output to OG-NaVi-BLAST2020.json
20:44:23 [INFO] Reading in JSON from OG-NaVi-BLAST2020.json
20:44:23 [INFO] JSON data loaded, available in the `json` attribute to parser
20:44:23 [INFO] Successfully parsed JSON output
20:44:23 [INFO] Successfully returned JSON output
20:44:23 [INFO] Parsed rounds to Pandas DataFrame
20:44:24 [INFO] Parsed kills to Pandas DataFrame
20:44:24 [INFO] Parsed damages to Pandas DataFrame
20:44:24 [INFO] Parsed grenades to Pandas DataFrame
20:44:24 [INFO] Parsed flashes to Pandas DataFrame
20:44:24 [INFO] Parsed weapon fires to Pandas DataFrame
20:44:24 [INFO] Parsed bomb_events to Pandas DataFrame
20:44:24 [INFO] Parsed frames to Pandas DataFrame
20:44:24 [INFO] Parsed player frames to Pandas D

dict_keys(['MatchId', 'ClientName', 'MapName', 'TickRate', 'PlaybackTicks', 'Rounds', 'Kills', 'Damages', 'Grenades', 'Flashes', 'WeaponFires', 'BombEvents', 'Frames', 'PlayerFrames'])

In [10]:
df["Kills"]

,Tick,Second,AttackerSteamID,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaID,...,ThruSmoke,Distance,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamID,Weapon,RoundNum,MatchId,MapName
0,39106,16.244094,7.656120e+16,mantuu,OGEsports,CT,-294.155731,626.777832,1.016636,1713.0,...,False,481.229305,False,None,None,NaN,USP-S,1,OG-NaVi-BLAST2020,de_dust2
1,39549,19.732283,7.656120e+16,ISSAA,OGEsports,CT,-1845.398804,2719.539307,32.390869,6887.0,...,False,1451.886671,False,None,None,NaN,USP-S,1,OG-NaVi-BLAST2020,de_dust2
2,39632,20.385827,7.656120e+16,mantuu,OGEsports,CT,-171.508743,761.183167,2.132068,1082.0,...,False,848.941885,False,None,None,NaN,USP-S,1,OG-NaVi-BLAST2020,de_dust2
3,39696,20.889764,7.656120e+16,flamie,Natus Vincere,T,-604.332642,1334.031250,-108.254135,1287.0,...,False,336.141751,False,None,None,NaN,Glock-18,1,OG-NaVi-BLAST2020,de_dust2
4,40045,23.637795,7.656120e+16,mantuu,OGEsports,CT,-149.031250,1001.231262,1.489925,1083.0,...,False,665.230508,True,Aleksib,OGEsports,7.656120e+16,USP-S,1,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,456139,56.866142,7.656120e+16,electronic,Natus Vincere,CT,-484.418335,1847.642334,-123.553299,5220.0,...,False,221.081781,False,None,None,NaN,M4A4,25,OG-NaVi-BLAST2020,de_dust2
158,458062,72.007874,7.656120e+16,valde,OGEsports,T,-2025.999146,1511.807251,33.910797,8063.0,...,False,602.142870,False,None,None,NaN,AK-47,25,OG-NaVi-BLAST2020,de_dust2
159,458223,73.275591,7.656120e+16,valde,OGEsports,T,-2015.828979,1507.713623,33.111588,8063.0,...,False,251.518662,False,None,None,NaN,AK-47,25,OG-NaVi-BLAST2020,de_dust2
160,458294,73.834646,7.656120e+16,electronic,Natus Vincere,CT,-1753.573730,1118.690674,31.989418,1245.0,...,False,467.218752,True,Boombl4,Natus Vincere,7.656120e+16,M4A4,25,OG-NaVi-BLAST2020,de_dust2
